# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5c9cde5610>
├── 'a' --> tensor([[0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160]])
└── 'x' --> <FastTreeValue 0x7f5c9ee49e50>
    └── 'c' --> tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                        [-0.4342, -0.7713,  0.4494,  0.0354],
                        [-0.3950,  0.4301, -0.1572, -0.3377]])

In [4]:
t.a

tensor([[0.8128, 0.3382, 0.2277],
        [1.8020, 0.8064, 1.2160]])

In [5]:
%timeit t.a

60 ns ± 0.243 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5c9cde5610>
├── 'a' --> tensor([[-0.7481,  0.9970,  1.1108],
│                   [-1.3213, -1.1791, -0.5648]])
└── 'x' --> <FastTreeValue 0x7f5c9ee49e50>
    └── 'c' --> tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                        [-0.4342, -0.7713,  0.4494,  0.0354],
                        [-0.3950,  0.4301, -0.1572, -0.3377]])

In [7]:
%timeit t.a = new_value

57.4 ns ± 0.042 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160]]),
    x: Batch(
           c: tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                      [-0.4342, -0.7713,  0.4494,  0.0354],
                      [-0.3950,  0.4301, -0.1572, -0.3377]]),
       ),
)

In [10]:
b.a

tensor([[0.8128, 0.3382, 0.2277],
        [1.8020, 0.8064, 1.2160]])

In [11]:
%timeit b.a

54.4 ns ± 0.606 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.4071, -0.3780,  0.3133],
               [-2.0053, -0.6846, -0.8265]]),
    x: Batch(
           c: tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                      [-0.4342, -0.7713,  0.4494,  0.0354],
                      [-0.3950,  0.4301, -0.1572, -0.3377]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.399 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

863 ns ± 0.485 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 29.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 228 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

141 µs ± 285 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5bf5f41640>
├── 'a' --> tensor([[[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]],
│           
│                   [[0.8128, 0.3382, 0.2277],
│                    [1.8020, 0.8064, 1.2160]]])
└── 'x' --> <FastTreeValue 0x7f5bf5f41970>
    └── 'c' --> tensor([[[ 1.1000,  0.0676, -1.7964, -1.8103],
           

In [21]:
%timeit t_stack(trees)

32.1 µs ± 36.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5bf874aaf0>
├── 'a' --> tensor([[0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160],
│                   [0.8128, 0.3382, 0.2277],
│                   [1.8020, 0.8064, 1.2160]])
└── 'x' --> <FastTreeValue 0x7f5bf874abb0>
    └── 'c' --> tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                        [-0.4342, -0.7713,  0.4494,  0.0354],
                        [-0.3950,  0.4301, -0.1572, -0.3377],
   

In [23]:
%timeit t_cat(trees)

30.1 µs ± 48.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.3 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]],
       
               [[0.8128, 0.3382, 0.2277],
                [1.8020, 0.8064, 1.2160]]]),
    x: Batch(
           c: tensor([[[ 1.1000,  0.0676, -1.7964, -1.8103],
                       [-0.4342, -0.7713,  0.4494,  0.0354],
                       [-0.3950,  0.4301, -0.1572, -0.3377]],
              
                      [[ 1.1000,  0.0676, 

In [26]:
%timeit Batch.stack(batches)

80.7 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160],
               [0.8128, 0.3382, 0.2277],
               [1.8020, 0.8064, 1.2160]]),
    x: Batch(
           c: tensor([[ 1.1000,  0.0676, -1.7964, -1.8103],
                      [-0.4342, -0.7713,  0.4494,  0.0354],
                      [-0.3950,  0.4301, -0.1572, -0.3377],
                      [ 1.1000,  0.0676, -1.7964, -1.8103],
                      [-0.4342, -0.7713,  0.4494,  0.0354],
                     

In [28]:
%timeit Batch.cat(batches)

151 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

325 µs ± 4.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
